<img src="../../thumbnail.png" width=250 alt="CESM LENS image"></img>

# Enhanced Intake-ESM Catalog Demo

---

## Overview
This notebook compares one [Intake-ESM](https://intake-esm.readthedocs.io/en/stable/) catalog with an enhanced version that includes additional attributes. Both catalogs are an inventory of the [NCAR Community Earth System Model (CESM) Large Ensemble (LENS) data hosted on AWS S3](https://doi.org/10.26024/wt24-5j82).

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Necessary | |

- **Time to learn**: 10 minutes

---

## Imports

In [ ]:
import intake
import pandas as pd

## Original Intake-ESM Catalog

At import time, the `intake-esm` plugin is available in `intake`’s registry as `esm_datastore` and can be accessed with `intake.open_esm_datastore()` function. 

In [ ]:
cat_url_orig = 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)
coll_orig

Here's a summary representation:

In [ ]:
print(coll_orig)

In an Intake-ESM catalog object, the `esmcat` class provides many useful attributes and functions. For example, we can get the collection's description:

In [ ]:
coll_orig.esmcat.description

We can also get the URL pointing to the catalog's underlying tabular representation:

In [ ]:
coll_orig.esmcat.catalog_file

That's a CSV file ... let's take a peek.

In [ ]:
df_orig = pd.read_csv(coll_orig.esmcat.catalog_file)
df_orig

However, we can save a step since an ESM catalog object provides a `df` instance which returns a dataframe too:

In [ ]:
df_orig = coll_orig.df
df_orig

Print out a sorted list of the unique values of selected columns

In [ ]:
for col in ['component', 'frequency', 'experiment', 'variable']:
    unique_vals = coll_orig.unique()[col]
    unique_vals.sort()
    count = len(unique_vals)
    print (col + ': ' ,unique_vals, " count: ", count, '\n')

## Finding Data

If you happen to know the meaning of the variable names, you can find what data are available for that variable. For example:

In [ ]:
df = coll_orig.search(variable='FLNS').df
df

We can narrow the filter to specific frequency and experiment:

In [ ]:
df = coll_orig.search(variable='FLNS', frequency='daily', experiment='RCP85').df
df

<div class="alert alert-warning"><b>The problem: </b>Do all potential users know that `FLNS` is a CESM-specific abbreviation for "net longwave flux at surface”? How would a novice user find out, other than by finding separate documentation, or by opening a Zarr store in the hopes that the long name might be recorded there? How do we address the fact that every climate model code seems to have a different, non-standard name for all the variables, thus making multi-source research needlessly difficult?</div>

<div class="alert alert-success"><b>The solution: </b><br>
    <h2> <i>Enhanced</i> Intake-ESM Catalog!</h2></div>

By adding additional columns to the Intake-ESM catalog, we should be able to improve semantic interoperability and provide potentially useful information to the users. Let's now open the *enhanced* collection description file:

<div class="alert alert-info"><b>Note: </b>The URL for the enhanced catalog differs from the original only in that it has <code>-enhanced</code> appended to <code>aws-cesm1-le</code></div>

In [ ]:
cat_url = 'https://ncar-cesm-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm1-le-enhanced.json'
coll = intake.open_esm_datastore(cat_url)
coll

As we did for the first catalog, let's obtain the `description` and `catalog_file` attributes.

In [ ]:
print(coll.esmcat.description) # Description of collection
print("Catalog file:", coll.esmcat.catalog_file)
print(coll) # Summary of collection structure

### Long names

In the catalog's representation above, note the addition of additional elements: `long_name`, `start`, `end`, and `dim`. Here are the first/last few lines of the enhanced catalog:

In [ ]:
df_enh = coll.df
df_enh

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    The <code>long_name</code>s are <em>not</em> <a href="https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html">CF Standard Names</a>, but rather are those documented at 
<a href="https://www.cesm.ucar.edu/projects/community-projects/LENS/data-sets.html">the NCAR LENS website</a>. For interoperability, the <code>long_name</code> column should be replaced by a <code>cf_name</code> column and possibly an <code>attribute</code> column to disambiguate if needed.
</div>

List all available variables by long name, sorted alphabetically:

In [ ]:
nameList = coll.unique()['long_name']
nameList.sort()
print(*nameList, sep='\n')

## Search capabilities

We can use an `intake-esm` catalog object's `search` function in several ways: 

Show all available data for a specific variable based on long name:

In [ ]:
myName = 'Salinity'
df = coll.search(long_name=myName).df
df

Search based on multiple criteria:

In [ ]:
df = coll.search(experiment=['20C','RCP85'], dim='3D', variable=['T','Q']).df
df

### Substring matches

In some cases, you may not know the exact term to look for. For such cases, inkake-esm supports searching for substring matches. With use of wildcards and/or regular expressions, we can find all items with a particular substring in a given column. Let’s search for:
- entries from experiment = ‘20C’
- all entries whose variable long name contains wind

In [ ]:
coll_subset = coll.search(experiment="20C", long_name="Wind*")

In [ ]:
coll_subset.df

If we wanted to search for Wind and wind, we can take advantage of [regular expression](https://docs.python.org/3/library/re.html) syntax to do so:

In [ ]:
coll_subset = coll.search(experiment="20C" , long_name="[Ww]ind*")

In [ ]:
coll_subset.df

### Other attributes

Other columns in the enhanced catalog may be useful. For example, the dimensionality column enables us to list all data from the ocean component that is 3D.

In [ ]:
df = coll.search(dim="3D",component="ocn").df
df

### Spatiotemporal filtering

If there were both regional and global data available (e.g., LENS and NA-CORDEX data for the same variable, both listed in same catalog), some type of coverage indicator (or columns for bounding box edges) could be listed.

Temporal extent in LENS is conveyed by the experiment (HIST, 20C, etc) but this is imprecise and requires external documentation. We have added start/end columns to the catalog, but Intake-ESM currently does not have built-in functionality to filter based on time.

We can do a simple search that exactly matches a temporal value:

In [ ]:
df = coll.search(dim="3D",component="ocn", end='2100-12').df
df

---

## Summary
In this notebook, we used Intake-ESM to explore a catalog of CESM LENS data. We then worked through some helpful features of the enhanced catalog.

### What's next?
We will use this data to recreate some figures from a paper published in [BAMS](https://www.ametsoc.org/index.cfm/ams/publications/bulletin-of-the-american-meteorological-society-bams/) that describes the CESM LENS project: {cite:t}`Kay:2015a`